<h1>DSCI 100 Final Project</h1>

<h1>Introduction</h1>

We’ve been tasked with performing data analysis to inform a study looking at how players interact with a server in the video game Minecraft. The study seeks to understand how players play video games, and how they interact with the virtual world around them. To that end, they need participants that will be engaged and present in their virtual world, in order to maximize the amount of data they are able to collect. 

In our analysis of their preliminary data, the broad question we’re trying to answer is: *We would like to know which "kinds" of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts*. To do that, we can analyze how different player characteristics affect the target variable we’re looking to maximize; time spent playing. The purpose of answering this question is to determine the characteristics the research group should look for in players when recruiting for their study. Our specific predictive question is: *How well can age, gender, subscription status, and experience level predict player's played hours?*

We have two datasets at our disposal: a table describing all players who participated, and a table describing each session any player logged. For our analysis, we’ve focused solely on the player dataset, as the session dataset is irrelevant to determining overall data contribution, and lacks any information about the players who recorded the sessions. So, that leaves us with the player dataset, which contains the following variables:
- Experience (object): Subjective player interpretation of their experience level, binned into categories. 
- Subscribed (bool): Boolean variable describing whether or not a player is subscribed to a minecraft newsletter
- hashedEmail (object): A player’s email, altered for anonymity.
- Played_hours (float64): # of hours played by this player
- Name (object): A player’s real life name 
- Gender (object): A player’s real life gender
- Age (int64): Reported player age
- individualId (float64): id of the player
- organizationName (float64): player's organization name

The data has some obvious flaws, owing to its self-reported nature. Certain ages, names, and experience levels are clearly not accurately reported, which muddies the data and makes it far more difficult to extract real conclusions. There are steps we can take to mitigate those detracting data points, but doing so may also remove some valuable true data. 

For our analysis, the response variable will be played_hours, and the explanatory variables will be age, gender, subscribe, and experience. We will build a model that use the explanatory variables to predict the Played_hours variable. Our model’s accuracy we depend completely on the quality and relationships of the data. By building an accurate model, we can then establish correlations between certain traits and higher hours played, answering our question of what type of player contributes the most data. 

The remainder of this report showcases and narrates our analysis, code, and findings related to this study.

In [1]:
### Run this cell before continuing.
import pandas as pd
import altair as alt
import numpy as np
from sklearn import set_config
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, FunctionTransformer, StandardScaler
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.metrics import mean_squared_error

# Output dataframes instead of arrays
set_config(transform_output="pandas")

In [2]:
# Import raw player data into python
url = "https://drive.google.com/uc?export=download&id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz"
player_data_unclean = pd.read_csv(url)

# Drop unecessary columns 
player_data = player_data_unclean.drop(['name','hashedEmail','individualId','organizationName'], axis=1) 

# Assign all genders other than 'Male' or 'Female' to the 'Other' category
player_data.loc[~player_data['gender'].isin(['Male', 'Female']), 'gender'] = 'Other'

In [3]:
# create 75%-25% train-test data split
training_df, testing_df = train_test_split(
    player_data,
    test_size=0.25,
    random_state=2000,
)

<h3>Exploratory Data Analysis</h3>

To start, let's generate some summary statistics of our training data.

In [4]:
# for numerical variables
training_df.describe()

,played_hours,age
count,147.000000,147.000000
mean,4.839456,21.102041
std,24.654260,8.628350
min,0.000000,8.000000
25%,0.000000,17.000000
50%,0.100000,19.000000
75%,0.600000,22.000000
max,223.100000,91.000000


In [5]:
# for categorical variables
training_df["gender"].value_counts()

gender
Male      93
Other     27
Female    27
Name: count, dtype: int64

In [6]:
training_df["subscribe"].value_counts()

subscribe
True     108
False     39
Name: count, dtype: int64

In [7]:
training_df["experience"].value_counts()

experience
Amateur     49
Veteran     33
Beginner    27
Regular     26
Pro         12
Name: count, dtype: int64

To visualize our training data, we will create bar charts to get a sense of the distributions of our predictor and response variables.

In [8]:
age_distribution = alt.Chart(training_df).mark_bar().encode(
    x=alt.X("age:Q", title="Age"),
    y=alt.Y('count()', title="Number of Players")
).properties(
    title="Fig 1: Distribution of Age"
)

played_hours_distribution = alt.Chart(training_df).mark_bar().encode(
    x=alt.X("played_hours:Q", title="Played Hours"),
    y=alt.Y('count()', title="Number of Players")
).properties(
    title="Fig 2: Distribution of Played Hours"
)

experience_distribution = alt.Chart(training_df).mark_bar().encode(
    x=alt.X("experience:N", title="Experience"),
    y=alt.Y('count()', title="Number of Players")
).properties(
    title="Fig 3: Distribution of Experience"
)

gender_distribution = alt.Chart(training_df).mark_bar().encode(
    x=alt.X("gender:N", title="Gender"),
    y=alt.Y('count()', title="Number of Players")
).properties(
    title="Fig 4: Distribution of Gender"
)

subscribe_distribution = alt.Chart(training_df).mark_bar().encode(
    x=alt.X("subscribe:N", title="Subscribe"),
    y=alt.Y('count()', title="Number of Players")
).properties(
    title="Fig 5: Distribution of Subscribe"
)
age_distribution | played_hours_distribution | experience_distribution | gender_distribution | subscribe_distribution

alt.HConcatChart(...)

For our numerical predictor variable, age, we will create a scatterplot against played_hours to observe if there are any linear patterns.

In [9]:
training_df['log_played_hours'] = np.log1p(training_df['played_hours'])

alt.Chart(training_df).mark_point(opacity=0.5).encode(
    x=alt.X('age:Q').title("Age"),
    y=alt.Y('log_played_hours:Q').title("Played Hours (log scale)"),
).properties(title='Fig 6: Played Hours vs Age')

alt.Chart(...)

For each categorical predictor variable, we can create bar plots to visualize how average played hours vary across categories.

In [10]:
average_played_hours_by_experience = alt.Chart(training_df).mark_bar().encode(
    x=alt.X("experience:N", title="Experience"),
    y=alt.Y('mean(played_hours):Q', title="Average Played Hours"),
    color=alt.Color("experience:N", legend=None)
).properties(
    title="Fig 7: Average Played Hours vs Experience"
)

average_played_hours_by_subscribe = alt.Chart(training_df).mark_bar().encode(
    x=alt.X("subscribe:N", title="Subscribe"),
    y=alt.Y('mean(played_hours):Q', title="Average Played Hours"),
    color=alt.Color("subscribe:N", legend=None)
).properties(
    title="Fig 8: Average Played Hours vs Subscribe"
)

average_played_hours_by_gender = alt.Chart(training_df).mark_bar().encode(
    x=alt.X("gender:N", title="Gender"),
    y=alt.Y('mean(played_hours):Q', title="Average Played Hours"),
    color=alt.Color("gender:N", legend=None)
).properties(
    title="Fig 9: Average Played Hours vs Gender"
)
average_played_hours_by_experience | average_played_hours_by_subscribe | average_played_hours_by_gender

alt.HConcatChart(...)

**Insights gained from summary and visualizations:**
- The distribution of age is heavily centered around 15-30 year olds. There are some outliers (age 90-100) that may effect our model if we choose to perform linear regression.
- The distribution of played_hours is skewed towards small values.
- There are more male players than players of other genders.
- While amateur players are the largest group, regular players play the most hours on average.
- Subscribed players spend more hours on average playing than non subscribed players.
- Female players have higher average played hours than male players.
- There is no obvious linear relationship between age and played hours. 

<h1>Method & Results</h1>

To assess our question of whether age, gender, subscription status, and experience level are good predictors of played hours, a regression model would be appropriate since played hours is a numerical variable. Our EDA showed no clear linear relationship between age and played hours, so we will use a K-NN Regression model. Therefore, no assumption about the underlying pattern of the data is needed. However, we are aware that a limitation of using KNN regression is that the predictions are less interpretable than linear regression coefficients. Now we can start the analysis and modelling.

In [11]:
# create X_train, y_train, X_test, y_test
X_train = training_df[["gender", "age", "experience", "subscribe"]]
y_train = training_df["played_hours"]
X_test = testing_df[["gender", "age", "experience", "subscribe"]]
y_test = testing_df["played_hours"]

For preprocessing, we will transform all of our categorical predictor variables to numerical values. Experience has an inherent hierarchy, so we will use ordinal encoding. Subscription is a boolean variable, so we will encode it as 0 (false) or 1 (true). As for Gender, there are more than two values, and there is no inherent ordering to them, so we will use one-hot encoding. As seen below, we will build a preprocessor to encapsulate all of these steps.

In [12]:
# building preprocessor 

# specify order for experience level
experience_order = [['Amateur', 'Beginner', 'Regular', 'Veteran', 'Pro']]

preprocessor = make_column_transformer(
    (OrdinalEncoder(dtype=int, categories=experience_order), ["experience"]),     # Map experience levels to integers using ordinal encoding
    (FunctionTransformer(lambda x: x.astype(int)), ["subscribe"]),                # Convert boolean subscription status to integers
    (OneHotEncoder(dtype=int, sparse_output=False), ["gender"]),                  # One hot encode the gender variable to create numerical dummy variables
    (StandardScaler(),["age"]),                                                   # Scaling would undermine the numerical mapping of gender/experience/subscribe
    remainder='passthrough',                                                     
    verbose_feature_names_out=False
)

To search for the best model, we will conduct hyperparameter tuning on the parameter n_neighbors with GridSearchCV and 5-fold cross-validation, searching over values of K from 1 to 50.

In [13]:
# creating pipeline with preprocessor and K-NN Regression model
pipeline = make_pipeline(
    preprocessor,
    KNeighborsRegressor()
)

# grid search
k_vals = {"kneighborsregressor__n_neighbors": range(1, 50, 1)} # we will search over values of k from 1 to 50

grid_model = GridSearchCV(
    estimator=pipeline,
    param_grid=k_vals,
    cv=5,
    scoring="neg_root_mean_squared_error"
)

# fit on training set
grid_model.fit(X_train, y_train)

/opt/conda/lib/python3.11/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ordinalencoder',
                                                                         OrdinalEncoder(categories=[['Amateur',
                                                                                                     'Beginner',
                                                                                                     'Regular',
                                                                                                     'Veteran',
                                                                                                     'Pro']],
                                                                                        dtype=<class 'int'>),
                                                                         ['experience']),
                                                                        ('functiontransformer',
                                                                         FunctionTransformer(func=<function <lambda> at 0x7f0999938ae0>),
                                                                         ['subscribe']),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(dtype=<class 'int'>,
                                                                                       sparse_output=False),
                                                                         ['gender']),
                                                                        ('standardscaler',
                                                                         StandardScaler(),
                                                                         ['age'])],
                                                          verbose_feature_names_out=False)),
                                       ('kneighborsregressor',
                                        KNeighborsRegressor())]),
             param_grid={'kneighborsregressor__n_neighbors': range(1, 50)},
             scoring='neg_root_mean_squared_error')

In [14]:
# Getting k value used by best model
best_k_val = grid_model.best_params_
best_k_val

{'kneighborsregressor__n_neighbors': 48}

In [15]:
# getting the RMSE for our best model
best_RMSE = -grid_model.best_score_
best_RMSE

np.float64(20.70253652829163)

The best model used a K value of 48 and achieved an RMSE of **20.70**

After we found the best model, we will predict on our test set, computing the RMSPE with our prediction and y_test. This process will determine how well our model used age, gender, subscribe, and experience to predict played_hours.

In [16]:
# predict on testing set
y_pred = grid_model.predict(X_test)

# compute the RMSPE with our prediction and y_test
prediction_score = mean_squared_error(y_pred, y_test)**0.5

prediction_score

np.float64(37.13418581410381)

Our final RMSPE on the test set achieved is **37.13**.

<h3>Visualizing of our model's predictions</h3>
Since our model is a multivariable K-NN regression with four predictor variables, we can not visualize the full model in two dimensions. Thus, to create a 2D plot, we will focus on only one predictor (age) and plotted age vs. played_hours. To overlay our predicted values, we will hold the other three predictor variables constant and then have our model generate predictions across a grid of age values, giving us a prediction curve that shows how the model’s estimated played_hours changes with age, given fixed values for the other variables.

In [17]:
# Create a grid of evenly spaced values along the range of the age data
prediction_grid = pd.DataFrame({
    "age": np.arange(testing_df["age"].min(), testing_df["age"].max(), 10)
})

# set the three categorical predictor variables to its most common value
prediction_grid["gender"] = "Male"
prediction_grid["subscribe"] = True
prediction_grid["experience"] = "Amateur"

# Predict the played hours for each of the age values in the grid
prediction_grid["predicted"] = grid_model.predict(prediction_grid)

# log scale the predicted and actual played hours for plotting
prediction_grid['log_predicted'] = np.log1p(prediction_grid['predicted'])
testing_df['log_played_hours'] = np.log1p(testing_df['played_hours'])

# create base plot of age vs played_hours
base_plot = alt.Chart(testing_df).mark_point(opacity=0.6).encode(
    x=alt.X("age")
        .scale(zero=False)
        .title("Age"),
    y=alt.Y("log_played_hours")
        .title("Played Hours (Log scale)")
)

# overlaying predicted values as a line
preds_plot = base_plot + alt.Chart(
    prediction_grid,
    title="Fig 10: Predicted played hours for subscribed, amateur male players with varying age, using KNN Regressor with K = 48"
).mark_line(
    color="#ff7f0e"
).encode(
    x="age",
    y="log_predicted"
)

preds_plot

alt.LayerChart(...)

<h1>Discussion</h1>

Using 5-fold cross-validation and performing hyperparameter turning with a K  range of 1-50, we found that the best model for our K-NN regression was K = 48 with an RMSE training score of 20.70. On the test dataset, our model achieved an RMSPE score of 37.13, indicating that, on average, our predictions deviate by around 37 hours from the actual played hours. Therefore, our model was unable to accurately predict the relationship between our predictor and response variables. Moreover, given that our model performed worse on the test dataset, we can conclude that it was unable to generalize well with unseen data. 

This poor result was somewhat expected given the limitations of our dataset and chosen prediction model. K-NN regression models tend to struggle with a large number of predictors. In addition, while discussing potential issues that could occur with using ordinal and one-hot encoded categorical variables in our K-NN regression is beyond the scope of DSCI100, we believe it is important to acknowledge that these factors could have impacted the results of our prediction model. 

However, the most notable limitation comes from our dataset, where nearly half of the observations in the players dataset recorded zero hours. Typically, underfitting and overfitting can occur when the number of K neighbours is too small or too large, which also causes the RMSPE to increase. Despite a reasonable number of K neighbours (48 neighbours for 196 observations in the players dataset), our prediction model was underfitting due to the large clusters of zero hours, making any other values insignificant in calculating the KNN regression and causing a high RMSPE score. Additionally, the large imbalance of zero hours played skewed our predicted hours downward, flattening and obscuring any meaningful patterns that could have been visualized between age and played hours in Figure 10. 
Given our findings, the team responsible for this dataset should be compelled to gather more observations from participants who actually played the game. For example, if a company used our model to understand the demographics of active players and that information was then used to guide marketing for game subscriptions, the resulting ads would likely target the wrong people and negatively impact the game company.

If my group were to continue our work with this dataset, it would be interesting to explore different predictive models and whether they could perform a more accurate prediction of played hours. Specifically, if there is a prediction tool that can address zero values in a dataset effectively, or if there is a different method to wrangle a dataset with a mix of categorical and numerical variables. Another interesting direction is to examine one particular predictor variable, like subscription, and ask if there is a causal relationship between increased subscription and participants playing longer. Or, given the difficulties our team faced due to the large amount of zero-played hours, we could pivot our analysis to a classification problem and identify who is more likely to contribute zero hours in a study.
